In [6]:
import pandas as pd
import json

In [7]:
apartments = pd.read_csv('../data/curated/geometry_property.csv')

In [8]:
tram_stops = pd.read_csv('../data/curated/tram_stop.csv')
train_stations = pd.read_csv('../data/curated/train_station.csv')
schools = pd.read_csv('../data/curated/school_locations.csv')
restaurants = pd.read_csv('../data/curated/melb_resto.csv')
landmarks = pd.read_csv('../data/curated/melb_landmarks.csv')

In [9]:
train_stations

,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,TICKETZONE,ROUTEUSSP,geometry
0,19970,Royal Park Railway Station (Parkville),-37.781193,144.952301,1,Upfield,POINT (144.95230710206772 -37.781179806470384)
1,19971,Flemington Bridge Railway Station (North Melbo...,-37.788140,144.939323,1,Upfield,POINT (144.93932910356588 -37.78812681562748)
2,19972,Macaulay Railway Station (North Melbourne),-37.794267,144.936166,1,Upfield,POINT (144.9361719034498 -37.79425383545)
3,19973,North Melbourne Railway Station (West Melbourne),-37.807419,144.942570,1,"Flemington,Sunbury,Upfield,Werribee,Williamsto...",POINT (144.9425759409124 -37.80740580596864)
4,19974,Clifton Hill Railway Station (Clifton Hill),-37.788657,144.995417,1,"Mernda,Hurstbridge",POINT (144.99542285377925 -37.78864386946561)
...,...,...,...,...,...,...,...
215,19965,Coburg Railway Station (Coburg),-37.742345,144.963336,1,Upfield,POINT (144.9633418243668 -37.742331846983326)
216,19966,Moreland Railway Station (Coburg),-37.754485,144.961823,1,Upfield,POINT (144.96182910331285 -37.75447185695532)
217,19967,Anstey Railway Station (Brunswick),-37.761242,144.960684,1,Upfield,POINT (144.96069006277764 -37.761228876919425)
218,19968,Brunswick Railway Station (Brunswick),-37.767721,144.959587,1,Upfield,POINT (144.9595927822653 -37.767707796884636)


In [10]:
# Calculate Haversine Distance
import math

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the earth in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c  # Distance in km
    return d

In [11]:
for index, row in apartments.iterrows():
    lon1 = row['longitude']
    lat1 = row['latitude']

    num_stops = 0 
    for index_stops, row2 in tram_stops.iterrows():
        lon2 = row2['LONGITUDE']
        lat2 = row2['LATITUDE']

        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 0.5:
            num_stops += 1

    apartments.at[index, 'num_stops'] = num_stops

In [12]:
for index, row in apartments.iterrows():
    lon1 = row['longitude']
    lat1 = row['latitude']
    
    num_stations = 0
    for index_stations, row3 in train_stations.iterrows():
        lon2 = row3['LONGITUDE']
        lat2 = row3['LATITUDE']
    
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 0.5:
            num_stations += 1
    apartments.at[index, 'num_stations'] = num_stations

In [13]:
for index, row in apartments.iterrows():
    lon1 = row['longitude']
    lat1 = row['latitude']
    
    num_schools = 0
    for index_schools, row4 in schools.iterrows():
        lon2 = row4['X']
        lat2 = row4['Y']
    
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 2:
            num_schools += 1
    apartments.at[index, 'num_schools'] = num_schools

In [14]:
for index, row in apartments.iterrows():
    lon1 = row['longitude']
    lat1 = row['latitude']
    
    num_restaurants = 0
    for index_restaurants, row5 in restaurants.iterrows():
        lon2 = row5['longitude']
        lat2 = row5['latitude']
    
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 1:
            num_restaurants += 1
    apartments.at[index, 'num_restaurants'] = num_restaurants

In [15]:
# Preprocess the 'co_ordinates' column to ensure it's valid JSON
landmarks['co_ordinates'] = landmarks['co_ordinates'].str.replace("'", "\"")
landmarks['co_ordinates'] = landmarks['co_ordinates'].apply(json.loads)

landmarks['lon'] = landmarks['co_ordinates'].apply(lambda x:x['lon'])
landmarks['lat'] = landmarks['co_ordinates'].apply(lambda x:x['lat'])

In [16]:
for index, row in apartments.iterrows():
    lon1 = row['longitude']
    lat1 = row['latitude']
    
    num_landmarks = 0
    for index_landmarks, row6 in landmarks.iterrows():
        lon2 = row6['lon']
        lat2 = row6['lat']
    
        distance = haversine(lat1, lon1, lat2, lon2)
        if distance <= 1:
            num_landmarks += 1
    apartments.at[index, 'num_landmarks'] = num_landmarks

In [22]:
apartments['total_transport'] = apartments['num_stops'] + apartments['num_stations']
apartments.sort_values(by='num_restaurants', ascending=False)

,Address,Suburb,Prices,Estimated resident population (no.),Population density (persons/km2),Median age - persons (years),Working age population (aged 15-64 years) (no.),Employed (no.),Unemployed (no.),Renter (no.),...,Count of homeless persons (no.),"Used at least one form of public transport (train, tram, bus, ferry) (no.)",latitude,longitude,num_stops,num_stations,num_schools,num_restaurants,num_landmarks,total_transport
4215,"3909/1 Queensbridge Square, \nSOUTHBANK VIC 3006",SOUTHBANK,725.0,16538.0,20569.7,30.9,14882.0,0.0,0.0,48.0,...,30.0,1844.0,-37.821212,144.961919,18.0,0.0,10.0,1127.0,66.0,18.0
909,"1109/7 Riverside Quay, \nSOUTHBANK VIC 3006",SOUTHBANK,620.0,16538.0,20569.7,30.9,14882.0,0.0,0.0,48.0,...,30.0,1844.0,-37.821256,144.964540,19.0,1.0,10.0,1081.0,62.0,20.0
1267,"3101/7 Riverside Quay, \nSOUTHBANK VIC 3006",SOUTHBANK,1449.0,16538.0,20569.7,30.9,14882.0,0.0,0.0,48.0,...,30.0,1844.0,-37.821256,144.964540,19.0,1.0,10.0,1081.0,62.0,20.0
621,"7801/7 Riverside Quay, \nSOUTHBANK VIC 3006",SOUTHBANK,1800.0,16538.0,20569.7,30.9,14882.0,0.0,0.0,48.0,...,30.0,1844.0,-37.821256,144.964540,19.0,1.0,10.0,1081.0,62.0,20.0
2000,"41/1 Riverside Quay, \nSOUTHBANK VIC 3006",SOUTHBANK,650.0,16538.0,20569.7,30.9,14882.0,0.0,0.0,48.0,...,30.0,1844.0,-37.821256,144.964540,19.0,1.0,10.0,1081.0,62.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564,"51/1 Graham Street, \nPORT MELBOURNE VIC 3207",PORT MELBOURNE,520.0,16373.0,5869.7,43.2,11248.0,0.0,0.0,46.0,...,44.0,379.0,-37.841867,144.943494,0.0,0.0,1.0,0.0,0.0,0.0
1565,"202/503 Plenty Road, \nPRESTON VIC 3072",PRESTON,520.0,33825.0,6306.2,75.0,24317.0,0.0,0.0,241.0,...,273.0,1434.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1566,"2/100 Madeline Street, \nPRESTON VIC 3072",PRESTON,520.0,33825.0,6306.2,75.0,24317.0,0.0,0.0,241.0,...,273.0,1434.0,-37.733168,145.018538,5.0,0.0,11.0,0.0,0.0,5.0
1567,"4/977-979 High Street, \nRESERVOIR VIC 3073",RESERVOIR,520.0,52065.0,10943.5,154.1,35092.0,0.0,0.0,490.0,...,416.0,1900.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Define the absolute path for saving the file
file_path = '../data/curated/places_property.csv'

# Save the DataFrame to the specified absolute path
apartments.to_csv(file_path, index=False)